# Data preparation for Bioimage.io upload

Load data from the NAS

In [4]:
from mitosplit_net.util import load_h5

data_path = '//lebnas1.epfl.ch/microsc125/Watchdog/MitoSplit-Net/Data/'

mito_data = load_h5(data_path, 'Mito')
drp1_data = load_h5(data_path, 'Drp1')


Loading //lebnas1.epfl.ch/microsc125/Watchdog/MitoSplit-Net/Data/Mito.h5
Converting to array

Loading //lebnas1.epfl.ch/microsc125/Watchdog/MitoSplit-Net/Data/Drp1.h5
Converting to array


Extract one frame and save as sample input

In [6]:
import numpy as np
from skimage.io import imsave

print(mito_data.shape)
frame_number = 5932

sample_input = np.array([mito_data[frame_number], drp1_data[frame_number]])
print(sample_input.shape)

imsave("sample_input.tif", sample_input)

(37000, 128, 128)
(2, 128, 128)
